# Image Caching Service

Creates vm's with a desired image across all nodes on all sites. Once the image is cached, the compute node will not need to download it next time it is used, greatly reducing boot times.

More info - https://docs.openstack.org/nova/ussuri/admin/image-caching.html

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Create a dictionary containing all sites and their host counts

In [ ]:
# Get a pandas.io.formats.style.Styler object of all sites and their info
try:
    site_list = fablib.list_sites(quiet=True)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Gather the list of sites and their respective host counts
site_name_list = site_list.data["Name"].tolist()
site_host_counts = site_list.data["Hosts"].tolist()

# Create dictionary of {site_name: host_count} using zip function
host_site_map = dict(zip(site_name_list, site_host_counts))

# Remove sites with 0 workers
host_site_map = {k: v for k, v in host_site_map.items() if v != 0}

# Print the dictionary
print(host_site_map)

## Step 4: Select an image
#### Query images to chose from

In [ ]:
print(fablib.get_image_names())

#### Set image to cache

In [ ]:
# Specify which image you would like to cache on all sites and their workers
image = "default_rocky_8"

### Summary of what will be created

In [ ]:
node_count = 0
site_count = len(host_site_map)
for site in host_site_map:
    node_count += host_site_map[site]
    
print(f'Prepared to create {site_count} slices with a total of {node_count} \"{image}\" nodes spanning accross:')
for site in host_site_map:
    print(f'{host_site_map[site]} workers on site {site}')

## Step 5: Cache image on all sites and their workers
1. Create slice for each site
2. Add a node with given image for each worker on site
3. Submit slice

In [ ]:
# Set to false to create slices in parallel
sequential_run = False

# Create a tag for this run (Each slice will end with this tag)
# Used for re-running without having to wait for all previous slices to finish deleting
tag = "1"

for site in host_site_map:
    # Confirm site has workers
    if host_site_map[site] == 0:
        break
        
    try:
        # Create a slice for current site
        # print(f'Creating slice {slice_name}')
        slice_name = f'{site}-slice-{tag}'
        slice = fablib.new_slice(name=slice_name)
        
        # Create a node for each worker on the site
        for worker_number in range (1, host_site_map[site] + 1):
            worker = f'{site.lower()}-w{worker_number}.fabric-testbed.net'
            node_name = f'node-{worker_number}'
            # Add node
            # print(f'- Adding node (name={node_name}, site={site}, host={worker}, image={image})')
            node = slice.add_node(name=node_name, site=site, host=worker, image=image)

        #Submit Slice Request
        slice.submit(wait=sequential_run)
    except Exception as e:
        print(f"Exception: {e}")

## Step 6: List slices to confirm everything worked

Checks the entire list of slice supposed to be created above so you can see slices that didn't get created and should have.

In [ ]:
for site in host_site_map:
    slice_name = f'{site}-slice-{tag}'
    try:
        slice = fablib.get_slice(slice_name)
        print(f'{slice_name} - {slice.get_state()}')
    except Exception as e:
        print(f"{slice_name} - Exception: {e}")

## Step 7: Delete all slices when finished

In [ ]:
for site in host_site_map:
    try:
        slice = fablib.get_slice(f'{site}-slice-{tag}')
        slice.delete()
    except Exception as e:
        print(f"Exception: {e}")